In [93]:
import pandas as pd
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from modules.cleaners import clean_text, simple_clean
import string
pd.options.mode.chained_assignment = None
import nltk
from nltk.stem import WordNetLemmatizer


In [94]:
#create dataframe with year, president and party columns
df = pd.read_csv('data/US presidents.csv', delimiter=';')

#rename column for easier reading
df.rename(columns={'Years (after inauguration)': 'year'}, inplace=True)

In [95]:
#Make a series with the filenames:
path = 'speeches/'
files = pd.Series(sorted(os.listdir(path))).map(lambda x: x[:-4])
files

0      1900
1      1901
2      1902
3      1903
4      1904
       ... 
116    2017
117    2018
118    2019
119    2020
120    2021
Length: 121, dtype: object

In [96]:
#method that returns dict with key: year (int) value: speech (str)
def get_text_dict():
    txtdict = {}

    for file in files:
        file_ = path + file + '.txt'
        f = open(file_, encoding='utf8', errors='ignore')
        filetxt = f.read()
        txtdict[int(file)] = filetxt

    return txtdict

In [97]:
text = get_text_dict()

In [98]:
#Add series to df with speeches from dict values
speech_list = list(text.values())
sp = pd.Series(speech_list)
df['speech'] = sp

# Cleaning round 1

In [99]:
first_clean = lambda x: simple_clean(x)

df['first_clean'] = df.speech.apply(first_clean)

In [100]:
first_clean_tokenized = lambda x: clean_text(x)

df['first_clean_tokenized'] = df.speech.apply(first_clean_tokenized)

In [101]:
#saving dataframe
df.to_pickle('pickled_data/data_first_clean.pkl')

# Cleaning round 2

In [102]:
#Creates a copy of dataframe for round 2:
df2 = df

In [103]:
def lemmatize(list):
    wordnet_lemmatizer = WordNetLemmatizer()
    wordlist = []
    for word in list:
        wordlist.append(wordnet_lemmatizer.lemmatize(word))
    return wordlist

In [104]:
df2['second_clean_lemmatized'] = df['first_clean_tokenized'].apply(lemmatize)
df2 = df2.drop(['first_clean', 'first_clean_tokenized'], axis=1)

In [106]:
#saving dataframe
df2.to_pickle('pickled_data/data_second_clean.pkl')